# 00. Preparation

## 00.1 install necessary and optional Python packages


`conda install cytoolz seaborn scikit-learn statsmodels numba pytables`

`conda install -c conda-forge scanpy python-igraph leidenalg`

`conda install -c conda-forge louvain multicore-tsne`

`pip3 install pyscenic`

## 00.2 Download auxiliary files and expression matrix (PBMC 10k from 10x Genomics)
`wget https://raw.githubusercontent.com/aertslab/pySCENIC/master/resources/hs_hgnc_tfs.txt`

`wget https://resources.aertslab.org/cistarget/motif2tf/motifs-v9-nr.hgnc-m0.001-o0.0.tbl`

`wget https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc9nr/gene_based/hg38__refseq-r80__10kb_up_and_down_tss.mc9nr.feather`

`wget http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_10k_v3/pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz`

`tar xvf pbmc_10k_v3_filtered_feature_bc_matrix.tar.gz`


# 01. Preprocessing

In [27]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import scanpy as sc
import loompy as lp

In [40]:
adata = sc.read_10x_mtx(path="./data/filtered_feature_bc_matrix/", var_names="gene_symbols")
adata.var_names_make_unique()

In [56]:
mito_genes_index = adata.var_names.str.startswith('MT-')
adata.obs['percent_mito'] = np.ravel(np.sum(adata[:, mito_genes_index].X, axis=1) / np.sum(adata.X, axis=1))
adata.obs['n_counts']     = np.ravel(adata.X.sum(axis=1))
adata.obs['n_genes']      = np.ravel( (adata.X > 0).sum(axis=1) )

In [59]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata = adata[adata.obs['n_genes']<4000, :]
adata = adata[adata.obs['percent_mito']<0.15, :]

In [71]:
import loompy as lp
row_attrs = {"Gene" : np.array(adata.var_names) }
col_attrs = {"CellID" : np.array(adata.obs_names),
             "nGene"  : np.array(adata.obs['n_genes']),
             "nUMI"   : np.array(adata.obs['n_counts'])
             }
lp.create("PBMC10k_filter.loom", adata.X.transpose(), row_attrs, col_attrs)

# 02. Network Inference

`pyscenic grn --number_workers 4 --output adj.tsv --method grnboost2 ./PBMC10k_filter.loom ./data/hs_hgnc_tfs.txt`

